In [1]:
import pkg_resources
from symspellpy import SymSpell, Verbosity
import nltk 
from nltk.corpus import wordnet
import pandas as pd
import numpy as np
import codecs
import pickle
import time
import clean_text2 as ct2

In [2]:
def read_data(file_name):
    with open(file_name,'r',encoding='utf8') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [29]:
data_set = pd.read_csv('covid_dataset_csv.csv')
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       667 non-null    object
 1   1       667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [30]:
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
data_set = pd.read_csv('covid_dataset_csv.csv',encoding='latin1')
vocab, w2v = read_data('glove.6B.50d.txt')
w2v['covid'] = w2v['coronavirus']
w2v['covid-19'] = w2v['coronavirus']
w2v['covid19'] = w2v['coronavirus']
w2v['corona virus'] = w2v['coronavirus']
all_words = w2v.keys()
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

Total Words in DataSet: 400000


True

In [31]:
def spelling_correction(sym_spell,input_term):
    suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)
    sentence = []
    print(len(suggestions))
    for suggestion in suggestions:
        sentence.append(str(suggestion).split(',')[0])
    return sentence

In [32]:
def cos_sim(u,v):
    numerator_ = u.dot(v)
    denominator_= np.sqrt(np.sum(np.square(u))) * np.sqrt(np.sum(np.square(v)))
    return numerator_/denominator_

In [56]:
def overall_similarity1(q1,data_set):
    similarity=[]
    q1_clean = [ct2.getClearReview(i) for i in [q1]][0].split()
    for i in range(data_set.shape[0]):
        q2 = data_set.iloc[i]['1']
#         print(q2)
        q2_clean = [ct2.getClearReview(i) for i in [q2]][0].split()
        ans = 0;
        for w1 in q1_clean:
            max1_ = 0
            for w2 in q2_clean:
                if w2 not in vocab:
                    continue
                sim1 = cos_sim(w2v[w1],w2v[w2])
                max1_ = max(sim1,max1_)
            ans += max1_
        similarity.append(ans/len(q1_clean))
    return similarity
def overall_similarity0(q1,data_set):
    similarity=[]
    q1_clean = [ct2.getClearReview(i) for i in [q1]][0].split()
    for i in range(data_set.shape[0]):
        q2 = data_set.iloc[i]['0']
#         print(q2)
        q2_clean = [ct2.getClearReview(i) for i in [q2]][0].split()
        ans = 0;
        for w1 in q1_clean:
            max1_ = 0
            for w2 in q2_clean:
                if w2 not in vocab:
                    continue
                sim1 = cos_sim(w2v[w1],w2v[w2])
                max1_ = max(sim1,max1_)
            ans += max1_
        similarity.append(ans/len(q1_clean))
    return similarity
def overall_similarity(q1,data_set):
    similarity0 = overall_similarity0(q1,data_set)
    similarity1 = overall_similarity1(q1,data_set)
    similarity = 2*np.array(similarity0)+np.array(similarity1)
#     similarity = np.array(similarity0)
#     print(similarity0.shape)
#     print(similarity1.shape)
    print(similarity.shape)
    similarity = list(similarity)
    return similarity

In [83]:
def answer(sym_spell,input_term):
#     input_term = str(spelling_correction(sym_spell,input_term)[0])
    input_term = input_term.replace('corona virus','coronavirus')
#     input_term = input_term.replace('ovid' , 'coronavirus')
    similarity = overall_similarity(input_term,data_set)
    print(similarity[np.argmax(similarity)])
    s = """We don\'t seem to have any information on that. Would you like to perform a web search instead?"""
    if similarity[np.argmax(similarity)]<2:
        return s
    return data_set.iloc[np.argmax(similarity)]['1']

In [84]:
print(answer(sym_spell,'What is melody so chocolaty?'))

(667,)
1.2912991414241597
We don't seem to have any information on that. Would you like to perform a web search instead?


In [73]:
f = open('symspell.pickle','wb')
pickle.dump(sym_spell, f)

In [74]:
f = open('symspell.pickle','rb')
sym_spell = pickle.load(f)

In [75]:
f = open('w2v.pickle','wb')
pickle.dump(w2v, f)

In [76]:
f = open('w2v.pickle','rb')
w2v = pickle.load(f)

In [77]:
answer(sym_spell,'What are causes of coronavirus?')

1


'can eating meat causes coronavirus ?'

In [79]:
f = open('dataset.pickle','wb')
pickle.dump(data_set, f)

In [80]:
f = open('dataset.pickle','rb')
dataset = pickle.load(f)

In [82]:
f = open('vocab.pickle','wb')
pickle.dump(vocab, f)

In [83]:
f = open('vocab.pickle','rb')
vocab = pickle.load(f)

In [14]:
print(str(spelling_correction(sym_spell,'hello')[0]))

1
hello
